## Load Embedded Data


In [1]:
import pandas
import pickle

MODEL ="dunzhang/stella_en_400M_v5" #dunzhang/stella_en_1.5B_v5
DIFFICULTY = "hard"
SAMPLE = 500

save_dir = f"embeddings/{MODEL}/{DIFFICULTY}/{SAMPLE}"

hotpot_qa_df = pandas.read_pickle(f'{save_dir}/df.pkl')
with open(f'{save_dir}/contexts.pkl', 'rb') as f:contexts = pickle.load(f)
with open(f'{save_dir}/similarity_matrix.pkl', 'rb') as f:similarity_matrix = pickle.load(f)

similarity_matrix = similarity_matrix.numpy()


# Agglomerative CLustering

In [2]:
from clustering.algorithms import Agglomerative
from clustering.visualization.counts import plot_cluster_counts

clusters  = Agglomerative(similarity_matrix,contexts,0.68)
plot_cluster_counts(clusters,
                    "Agglomerative Clustering",
                    "Cluster No.",
                    "No. of Items in Cluster")

In [3]:
from typing import Dict
from clustering.graph.create_from_clusters import ClusterGraph
import networkx as nx

cluster_graph = ClusterGraph(clusters=clusters,similarity_matrix=similarity_matrix)

individual_cluster_graphs:Dict[int, nx.Graph] = cluster_graph.get_individual_cluster_graphs()
connected_graph:nx.Graph = cluster_graph.get_connected_graph()

In [11]:
import plotly

from clustering.graph.visualization.graph_plots import get_individual_cluster_graph_plots
from clustering.graph.visualization.graph_plots import get_connected_graph_plot

graph_plots:Dict[int,plotly.graph_objs.Figure] = get_individual_cluster_graph_plots(individual_cluster_graphs)
cluster_plot:plotly.graph_objs.Figure = get_connected_graph_plot(connected_graph,clusters)

In [9]:
import pickle

with open(f'clusters/graph_plots.pkl', 'wb') as file:
    pickle.dump(graph_plots, file)

with open(f'clusters/cluster_plot.pkl', 'wb') as file:
    pickle.dump(cluster_plot, file)

# python cluster_plots.py --cluster_plot_dir="clusters/cluster_plot" --graph_plots_dir="clusters/graph_plots"

In [ ]:
from clustering.visualization.cluster_plots import ClusterVisualization

plot = ClusterVisualization(cluster_plot,graph_plots)
plot.run()

In [ ]:
from scipy.cluster import hierarchy

# Create a linkage matrix
Z = hierarchy.linkage(dissimilarity_matrix, method='single')
print("done")
# Create a dendrogram
fig = plt.figure(figsize=(10, 5))

In [ ]:
GROQ_API_KEY = "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"

In [ ]:

dn = hierarchy.dendrogram(Z)